In [1]:
import requests
from bs4 import BeautifulSoup
import os
import time
import random

### 得到每一页的全部项目链接（每10页操作一下）

In [2]:
headers = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'
}

In [11]:
import pickle
url_save_path = 'ArchDailyURLs.pickle'
title_save_path = 'ArchDailyTitles.pickle'

# 定义目标URL（"办公建筑"分类页面）
url_root = "https://www.archdaily.com"

proj_links_all = []
proj_titles_all = []

count=0
for i in range(0,92):
    # 每一页
    url = "https://www.archdaily.com/category/office-buildings/page/"+str(i+1)
    print(url)
    
    try:
        response = requests.get(url,headers=headers,timeout=60)
        content = response.content
        soup = BeautifulSoup(content, "html.parser")

        # 每一页各个项目链接
        proj_links = []
        proj_titles = []
        for li in soup.find_all('li',class_="afd-search-list__4xitem"):
            a = li.find('a', class_="afd-search-list__link")
            for a in li.find_all('a', class_="afd-search-list__link"):
                input_string = a['href']
                proj_title = a['href'].split("/")[2]
                url_proj = url_root + a['href']
                proj_links.append(url_proj)
                proj_titles.append(proj_title)
                count = count+1
        proj_links_all.append(proj_links)
        proj_titles_all.append(proj_titles)
        print("page "+str(i+1)+"   total projects num: "+str(count))
        time.sleep(1)
    except requests.exceptions.RequestException as e:
        print(f"Error downloading projects: {e}")
        continue

with open(url_save_path, 'wb') as f:
    pickle.dump(proj_links_all, f)
with open(title_save_path, 'wb') as f:
    pickle.dump(proj_titles_all, f)

https://www.archdaily.com/category/office-buildings/page/1
page 1   total projects num: 24
https://www.archdaily.com/category/office-buildings/page/2
page 2   total projects num: 48
https://www.archdaily.com/category/office-buildings/page/3
page 3   total projects num: 72
https://www.archdaily.com/category/office-buildings/page/4
page 4   total projects num: 96
https://www.archdaily.com/category/office-buildings/page/5
page 5   total projects num: 120
https://www.archdaily.com/category/office-buildings/page/6
page 6   total projects num: 144
https://www.archdaily.com/category/office-buildings/page/7
page 7   total projects num: 168
https://www.archdaily.com/category/office-buildings/page/8
page 8   total projects num: 192
https://www.archdaily.com/category/office-buildings/page/9
page 9   total projects num: 216
https://www.archdaily.com/category/office-buildings/page/10
page 10   total projects num: 240
https://www.archdaily.com/category/office-buildings/page/11
page 11   total projec

https://www.archdaily.com/category/office-buildings/page/88
page 88   total projects num: 2112
https://www.archdaily.com/category/office-buildings/page/89
page 89   total projects num: 2136
https://www.archdaily.com/category/office-buildings/page/90
page 90   total projects num: 2160
https://www.archdaily.com/category/office-buildings/page/91
page 91   total projects num: 2184
https://www.archdaily.com/category/office-buildings/page/92
page 92   total projects num: 2205


### 依次访问各个项目的链接

In [12]:
with open('ArchDailyURLs.pickle', 'rb') as f:
    proj_links_all = pickle.load(f)
with open('ArchDailyTitles.pickle', 'rb') as f:
    proj_titles_all = pickle.load(f)

In [13]:
import re

In [14]:
headers_proj = {
    'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36 Edg/115.0.1901.203'
}

In [20]:
problem_link= {}
for i in range(60,76):
    for j in range(0, len(proj_links_all[i])):
        title = proj_titles_all[i][j]
        link = proj_links_all[i][j]
        os.makedirs("../../../2_data/ArchDailyImages/"+title, exist_ok=True)
        try:
            project_response = requests.get(link,headers = headers_proj,timeout=60)
            project_content = project_response.content
            project_soup = BeautifulSoup(project_content, "html.parser")

            # 项目信息
            specs_li = project_soup.find_all("li", class_='afd-specs__item')
            specs_data = {}
            for li in specs_li:
                span_key =  li.find("span", class_='afd-specs__key')
                span_value =  li.find("span", class_='afd-specs__value')
                if span_key is not None:
                    key = re.sub(r'[\s\n]', '', span_key.get_text())
                    if span_value is not None:
                        value = re.sub(r'[\s\n]', '', span_value.get_text())
                        specs_data[key] = value
                    else:
                        specs_data[key] = ''
            # 保存为JSON文件
            specs_path = "../../../2_data/ArchDailyImages/"+title+"/specs.json"
            with open(specs_path, "w") as json_file:
                json.dump(specs_data, json_file)

            # 项目图片
            proj_pic = project_soup.find_all('a', class_='gallery-thumbs-link')
            img_tags = []
            for k in range(0,len(proj_pic)):
                p = proj_pic[k]
                img =  p.find('img')
                img_tag = img['alt'].split("- ")[-1]
                tag = img_tag.split(", ")
                img_tags.append(tag)

                img_link = img['src'].replace('medium_jpg', 'slideshow')
                try:
                    image_response = requests.get(img_link,headers = headers_proj,timeout=60)
                    image_data = image_response.content
                    with open(os.path.join("../../../2_data/ArchDailyImages",title, f"{k+1}.jpg"), "wb") as f:
                        f.write(image_data)
                    time.sleep(0.8) #存图间隔
                except requests.exceptions.RequestException as e:
                    problem_link[title] = link
                    print(f"Error downloading image: {e}")
                    continue
            tags_path = "../../../2_data/ArchDailyImages/"+title+"/img_tags.txt"
            # 将列表整理成指定格式的字符串
            formatted_data = ""
            for sublist in img_tags:
                formatted_data += ",".join(sublist) + "\n"

            # 将整理后的数据写入到本地文件
            with open(tags_path, "w") as file:
                file.write(formatted_data)

            print("page "+str(i+1)+" -- "+title+" saved. | specs: "+str(len(specs_data))+" | imgs: "+str(len(proj_pic)))
            time.sleep(1) #打开项目间隔
        except requests.exceptions.RequestException as e:
            problem_link[title] = link
            print("Error processing at page: "+str(i+1)+" project: "+str(j+1)+" -- "+title)
            continue

print(problem_link)

page 61 -- office-conversion-in-cascais-ggllatelier saved. | specs: 9 | imgs: 16
page 61 -- an-old-breton-barn-converted-into-an-artist-studio-modal-architecture saved. | specs: 3 | imgs: 35
page 61 -- cubic-buildings-by-nadaaa-and-himma-studio-redefine-the-office-space saved. | specs: 0 | imgs: 25
page 61 -- schmidt-hammer-lassen-wins-competition-to-design-mixed-use-complex-in-central-oslo saved. | specs: 0 | imgs: 8
page 61 -- city-of-paris-approves-mvrdv-s-restructuring-of-montparnasse-superblock saved. | specs: 0 | imgs: 4
page 61 -- office-in-son-bugadelles-ripolltizon saved. | specs: 13 | imgs: 18
page 61 -- nestle-waters-beirut-headquarters-bernard-mallat-architects-walid-zeidan saved. | specs: 14 | imgs: 30
page 61 -- valencia-dorte-mandrup-arkitekter saved. | specs: 9 | imgs: 25
page 61 -- miranda-law-firm-rita-pinto-ribeiro saved. | specs: 12 | imgs: 37
page 61 -- planning-korea-turns-to-the-microbial-world-to-envision-the-future-of-paris saved. | specs: 0 | imgs: 11
page 61 

page 64 -- magma-towers-glr-arquitectos saved. | specs: 12 | imgs: 30
page 64 -- novartis-campus-weiss-manfredi saved. | specs: 14 | imgs: 18
page 65 -- the-yellow-building-allford-hall-monaghan-morris saved. | specs: 12 | imgs: 15
page 65 -- unstudio-revamps-hanwha-offices-with-shimmering-led-facade saved. | specs: 0 | imgs: 12
page 65 -- silos-13-vib-architecture saved. | specs: 16 | imgs: 46
page 65 -- tbwa-lisbon-office-colectivarquitectura saved. | specs: 9 | imgs: 18
page 65 -- eso-headquarters-extension-auer-weber saved. | specs: 13 | imgs: 9
page 65 -- schuurman-group-bekkering-adams-architects saved. | specs: 4 | imgs: 20
page 65 -- mk-ck5-production-office-agaligo-studio saved. | specs: 11 | imgs: 52
page 65 -- foster-partners-designs-headquarters-for-copper-company-in-russia saved. | specs: 0 | imgs: 0
page 65 -- the-factory-pepe-gascon saved. | specs: 9 | imgs: 10
page 65 -- refurbishment-of-office-building-moura-martins-architects saved. | specs: 11 | imgs: 36
page 65 -- b

page 68 -- de-burgemeester-studioninedots saved. | specs: 9 | imgs: 27
page 69 -- trafag-e2a saved. | specs: 2 | imgs: 21
page 69 -- federal-center-south-building-1202-zgf-architects saved. | specs: 20 | imgs: 30
page 69 -- 2722-building-daniel-martin-cornejo saved. | specs: 9 | imgs: 38
page 69 -- cthb-law-office-salon-architects saved. | specs: 4 | imgs: 15
page 69 -- syd-energi-headquarters-gpp-arkitekter saved. | specs: 3 | imgs: 14
page 69 -- surfrider-foundation-gardera-d-architecture saved. | specs: 4 | imgs: 17
page 69 -- salas-regionales-del-golfo-mauricio-garcia-cue saved. | specs: 4 | imgs: 26
page 69 -- agencia-andaluza-ruiz-larrea-y-asociados saved. | specs: 4 | imgs: 13
page 69 -- administrative-centre-zwevegem-buro-ii-and-archi-i-and-sileghem-and-partners saved. | specs: 3 | imgs: 25
page 69 -- sanibell-roosros-architecten saved. | specs: 3 | imgs: 21
page 69 -- storebrand-offices-link-arkitektur saved. | specs: 4 | imgs: 8
page 69 -- fameline-properties-vardastudio-arch

page 72 -- corporate-office-building-for-hispasat-technology-center-herreros-arquitectos saved. | specs: 15 | imgs: 15
page 73 -- pavilions-in-corporate-campus-apeldoorn-adp-architects saved. | specs: 4 | imgs: 17
page 73 -- artis-headquarters-roswag-architekten saved. | specs: 11 | imgs: 20
page 73 -- premier-campus-office-in-kagithane-winning-proposal-jds-architects saved. | specs: 0 | imgs: 47
page 73 -- hase-bldg-8-c-a-coelacanth-and-associates saved. | specs: 4 | imgs: 28
page 73 -- one-world-trade-center-will-soon-top-out-at-1-776-feet saved. | specs: 0 | imgs: 6
page 73 -- one-yonge-hariri-pontarini-architects saved. | specs: 0 | imgs: 7
page 73 -- stardom-entertainment-office-d-lim-architects saved. | specs: 15 | imgs: 27
page 73 -- updated-plans-released-for-foster-partners-apple-campus-in-cupertino saved. | specs: 0 | imgs: 0
page 73 -- henning-larsen-architects-designs-new-danish-headquarters-for-microsoft saved. | specs: 0 | imgs: 9
page 73 -- shanghai-roche-master-plan-and

### 有问题的网页重新爬取

In [21]:
for title in problem_link:
    link = problem_link[title]
    os.makedirs("../../../2_data/ArchDailyImages/"+title, exist_ok=True)
    try:
        project_response = requests.get(link,headers=headers_proj,timeout=60)
        project_content = project_response.content
        project_soup = BeautifulSoup(project_content, "html.parser")

        # 项目信息
        specs_li = project_soup.find_all("li", class_='afd-specs__item')
        specs_data = {}
        for li in specs_li:
            span_key =  li.find("span", class_='afd-specs__key')
            span_value =  li.find("span", class_='afd-specs__value')
            if span_key is not None:
                key = re.sub(r'[\s\n]', '', span_key.get_text())
                if span_value is not None:
                    value = re.sub(r'[\s\n]', '', span_value.get_text())
                    specs_data[key] = value
                else:
                    specs_data[key] = ''
        # 保存为JSON文件
        specs_path = "../../../2_data/ArchDailyImages/"+title+"/specs.json"
        with open(specs_path, "w") as json_file:
            json.dump(specs_data, json_file)

        # 项目图片
        proj_pic = project_soup.find_all('a', class_='gallery-thumbs-link')
        img_tags = []
        for k in range(0,len(proj_pic)):
            p = proj_pic[k]
            img =  p.find('img')
            img_tag = img['alt'].split("- ")[-1]
            tag = img_tag.split(", ")
            img_tags.append(tag)

            img_link = img['src'].replace('medium_jpg', 'slideshow')
            try:
                image_response = requests.get(img_link,headers=headers_proj,timeout=60)
                image_data = image_response.content
                with open(os.path.join("../../../2_data/ArchDailyImages",title, f"{k+1}.jpg"), "wb") as f:
                    f.write(image_data)
                time.sleep(0.8) #存图间隔
            except requests.exceptions.RequestException as e:
                print(f"Error downloading image: {e}")
                continue
        tags_path = "../../../2_data/ArchDailyImages/"+title+"/img_tags.txt"
        # 将列表整理成指定格式的字符串
        formatted_data = ""
        for sublist in img_tags:
            formatted_data += ",".join(sublist) + "\n"

        # 将整理后的数据写入到本地文件
        with open(tags_path, "w") as file:
            file.write(formatted_data)

        print(title+" saved. | specs: "+str(len(specs_data))+" | imgs: "+str(len(proj_pic)))
        time.sleep(1) #打开项目间隔
    except requests.exceptions.RequestException as e:
        print("Error processing at "+title)
        continue

ampera-six-chrystalline-architect saved. | specs: 18 | imgs: 55
daum-space-1-mass-studies saved. | specs: 12 | imgs: 51


### 对每一个项目链接再次发送请求，解析项目信息和图片

#### 项目信息

In [16]:
project_link = "https://www.archdaily.com/1005735/the-white-renovation-office-building-olgooco"
project_response = requests.get(project_link)
project_content = project_response.content
project_soup = BeautifulSoup(project_content, "html.parser")

# 项目信息
specs_li = project_soup.find_all("li", class_='afd-specs__item')
data = {}
for li in specs_li:
    span_key =  li.find("span", class_='afd-specs__key')
    span_value =  li.find("span", class_='afd-specs__value')
    if span_key is not None:
        key = re.sub(r'[\s\n]', '', span_key.get_text())
        if span_value is not None:
            value = re.sub(r'[\s\n]', '', span_value.get_text())
            data[key] = value
        else:
            data[key] = ''
print(data)

{'Architects:': 'Olgooco', 'Area:': '3000m²', 'Year:': '2022', 'Photographs:': 'MohammadHassanEttefagh,ParhamTaghioff', 'Manufacturers:IvanSystem,SalcoCompany': '', 'LeadArchitects:': 'MehranKhoshroo', 'Contractor:': 'CandoEngineeringGroup', 'DesignTeam:': 'ReyhaneMiraftab-SaminMostafaei-SoroushZeydanian', 'Structure:': 'AfshinMasoudi', 'Electrical:': 'Mr.Mardani', 'Mechanical:': 'ImanEILBeigi', 'Graphic:': 'MahsaEsmaeili', 'Program/Use/BuildingFunction:': 'Officebuilding', 'Client:': 'PayaWealthManagement', 'Supervision:': 'OlgooOffice', 'City:': 'Tehran', 'Country:': 'Iran'}


#### 项目图片

In [76]:
# 项目介绍文本
paras = project_soup.find_all("p",{'class_': ''})
print(paras)

[<p class="afd-mobile-navigation__item-title">
Projects
</p>, <p class="afd-mobile-navigation__item-title">
Images
</p>, <p class="afd-mobile-navigation__item-title">
Products
</p>, <p class="afd-mobile-navigation__item-title">
Folders
</p>, <p class="afd-mobile-navigation__item-title avatar-title">
Feed
</p>, <p class="js-tooltip-text"></p>, <p class="thumbs afd-desktop-e clearfix"><a class="thumbs__link" data-nr-picture-id="64e3a7a3fbc6582d9274f21b" href="/1005735/the-white-renovation-office-building-olgooco/64e3a7a3fbc6582d9274f21b-the-white-renovation-office-building-olgooco-photo" rel="attachment"><img alt="The White Renovation Office Building / Olgooco - Interior Photography, Chair" class="thumbs__img b-lazy" data-nr-picture-id="64e3a7a3fbc6582d9274f21b" data-pin-nopin="true" data-src="https://images.adsttc.com/media/images/64e3/a7a3/fbc6/582d/9274/f21b/thumb_jpg/the-white-renovation-office-building-olgooco_13.jpg?1692641219" height="125" longdesc="https://www.archdaily.com/10057